In [5]:
import numpy as np
from lab3_proto import *
from lab3_tools import *
import sys  
sys.path.append('/home/tmh/Documents/yhu/speech/Speech-Speaker-Recognition/lab1')
from lab1_proto import *
sys.path.append('/home/tmh/Documents/yhu/speech/Speech-Speaker-Recognition/lab2')
from lab2_tools import *
from lab2_proto import *
from prondict import *

In [4]:
path2info('tidigits/disc_4.1.1/tidigits/train/man/ae/1a.wav')

('man', 'ae', '1', 'a')

In [5]:
loadAudio('tidigits/disc_4.1.1/tidigits/train/man/ae/z9z6531a.wav')

(array([11, 13, 11, ...,  9,  9,  9], dtype=int16), 20000)

## Target Class Definition

In [10]:
statelist = []
phoneHMMs = np.load('/home/tmh/Documents/yhu/speech/Speech-Speaker-Recognition/lab2/lab2_models_all.npz',allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone:phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' +str(id) for ph in phones for id in range(nstates[ph])]
statelist = stateList 
stateList

['ah_0',
 'ah_1',
 'ah_2',
 'ao_0',
 'ao_1',
 'ao_2',
 'ay_0',
 'ay_1',
 'ay_2',
 'eh_0',
 'eh_1',
 'eh_2',
 'ey_0',
 'ey_1',
 'ey_2',
 'f_0',
 'f_1',
 'f_2',
 'ih_0',
 'ih_1',
 'ih_2',
 'iy_0',
 'iy_1',
 'iy_2',
 'k_0',
 'k_1',
 'k_2',
 'n_0',
 'n_1',
 'n_2',
 'ow_0',
 'ow_1',
 'ow_2',
 'r_0',
 'r_1',
 'r_2',
 's_0',
 's_1',
 's_2',
 'sil_0',
 'sil_1',
 'sil_2',
 'sp_0',
 't_0',
 't_1',
 't_2',
 'th_0',
 'th_1',
 'th_2',
 'uw_0',
 'uw_1',
 'uw_2',
 'v_0',
 'v_1',
 'v_2',
 'w_0',
 'w_1',
 'w_2',
 'z_0',
 'z_1',
 'z_2']

In [8]:
stateList.index('ay_2')

8

## Forced Alignment

In [22]:
filename ='tidigits/disc_4.1.1/tidigits/train/man/nw/z43a.wav'
samples, samplingrate = loadAudio(filename)
lmcc = mfcc(samples)
print(lmcc.shape)

(178, 13)


In [3]:
wordTrans = list(path2info(filename)[2])
wordTrans

['z', '4', '3']

In [4]:
phoneTrans = words2phones(wordTrans,prondict)
phoneTrans

['sil',
 'z',
 'iy',
 'r',
 'ow',
 'sp',
 'f',
 'ao',
 'r',
 'sp',
 'th',
 'r',
 'iy',
 'sp',
 'sil']

In [15]:
utteranceHMM = concatHMMs(phoneHMMs,phoneTrans)
#utteranceHMM 

In [11]:
stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans
                for stateid in range(nstates[phone])]

In [19]:
print(stateTrans)

['sil_0', 'sil_1', 'sil_2', 'z_0', 'z_1', 'z_2', 'iy_0', 'iy_1', 'iy_2', 'r_0', 'r_1', 'r_2', 'ow_0', 'ow_1', 'ow_2', 'sp_0', 'f_0', 'f_1', 'f_2', 'ao_0', 'ao_1', 'ao_2', 'r_0', 'r_1', 'r_2', 'sp_0', 'th_0', 'th_1', 'th_2', 'r_0', 'r_1', 'r_2', 'iy_0', 'iy_1', 'iy_2', 'sp_0', 'sil_0', 'sil_1', 'sil_2']


## Align 